In [1]:
import pytz
from datetime import datetime, timedelta
import os
import geopandas as gpd
import json
from shapely.geometry import Polygon, Point, MultiPolygon
import shapefile
import re
import shapefile as shp  # Requires the pyshp package
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors
import numpy as np
import random
random.seed(10)

In [2]:
shapefile = gpd.read_file("../geo/geo_export_5bb8636f-65b7-450a-8fd9-7f01027fd84b.shp")
chicago_shape = shapefile["geometry"][0]

In [3]:
os.listdir('./data/')

['250000-tweets-2021-09-25_07-33-30.json',
 '250000-tweets-2021-09-23_05-03-28.json',
 '250000-tweets-2021-09-24_22-34-46.json',
 '250000-tweets-2021-09-24_20-03-20.json',
 '250000-tweets-2021-09-24_13-04-37.json',
 '250000-tweets-2021-09-24_15-16-26.json',
 '250000-tweets-2021-09-24_17-36-05.json',
 '250000-tweets-2021-09-24_10-59-07.json',
 '250000-tweets-2021-09-25_04-59-49.json',
 '250000-tweets-2021-09-25_01-48-23.json']

In [4]:
## get the twitter in chicago
## City scale analysis
## This block of code will takes a long time
## We iterate through all the twitter collected for find twitter in chicago
## Get the filename
filelist = os.listdir('./data/')
filelist

twitter_in_chicago = []

# Opening JSON file
for filename in filelist:
    filepath = "./data/"+filename
    print(filepath)
    f = open(filepath)
    data = json.load(f)
    
    ## Read the data if the centroid of the twitter point polygon lies within the boundary of the city of Chicago
    for i in range(0, len(data)):
        try:
            ##Need to deal with case when the shapefile is too big
            text = data[i]["text"]
            t = data[i]['created_at']
            ## Case 1
            ## Twitter with exact geospatial location
            if (data[i]['geo']!=None):
                lat = data[i]['geo']['coordinates'][0]
                lon = data[i]['geo']['coordinates'][1]
                exact_loc = Point(lon, lat)
                if chicago_shape.contains(point):
                    ## print("inside")
                    twitter_in_chicago.append((exact_loc, t, text))
            else:
                ## Twitter with a polygon bounding box
                poly = data[i]['place']['bounding_box']["coordinates"][0]
                lon = -1000
                lat = -1000

                lon = [p[0] for p in poly]
                lat = [p[1] for p in poly]
                centroid = (sum(lon) / len(poly), sum(lat) / len(poly))
                point = Point(centroid)
                ## check if a centroid is in the bounding box of chicago
                if chicago_shape.contains(point):
                    ## print("inside")
                    twitter_in_chicago.append((poly, t, text))
        except:
            ## no geographical location
            pass
    # Closing file
    f.close()

./data/250000-tweets-2021-09-25_07-33-30.json
./data/250000-tweets-2021-09-23_05-03-28.json
./data/250000-tweets-2021-09-24_22-34-46.json
./data/250000-tweets-2021-09-24_20-03-20.json
./data/250000-tweets-2021-09-24_13-04-37.json
./data/250000-tweets-2021-09-24_15-16-26.json
./data/250000-tweets-2021-09-24_17-36-05.json
./data/250000-tweets-2021-09-24_10-59-07.json
./data/250000-tweets-2021-09-25_04-59-49.json
./data/250000-tweets-2021-09-25_01-48-23.json


In [5]:
print ("There are in total "+str(len(twitter_in_chicago))+" geo-tagged Twitter Collected in Chicago")

There are in total 10629 geo-tagged Twitter Collected in Chicago


In [6]:
## Read the word heat dictionary
f = open('../geo/data20000.txt','r')
content = f.read()
f.close()
dict_word = {}
content_list = content.split(",")
for i in range(0,len(content_list)):
    try:
        word = content_list[i].split(":")[0].split("'")[1]
        #print(content_list[i].split(":"))
        val = float(content_list[i].split(":")[1])
        dict_word[word] = val
    except:
        pass

In [7]:
## Iterate through all twitter data in chicago

d_twitter = []
for i in range(0, len(twitter_in_chicago)):
    loc = twitter_in_chicago[i][0]
    t = twitter_in_chicago[i][1]
    text = twitter_in_chicago[i][2]
    res = re.findall(r'\w+', text.lower())
    val = 0
    for word in res:
        if word in dict_word.keys():
            val = val + dict_word[word]
    ## remove weather-irrelevant twitter
    ## if none of the word in the heat dictionary show up 
    if (val!=0):
        d_twitter.append((loc, t, val))

In [8]:
print("There are "+str(len(d_twitter))+" weather-related Twitter in Chicago")

There are 433 weather-related Twitter in Chicago


In [9]:
m_dic = {}
m_dic['Jan'] = 1
m_dic['Feb'] = 2
m_dic['Mar'] = 3
m_dic['Apr'] = 4
m_dic['May'] = 5
m_dic['Jun'] = 6
m_dic['Jul'] = 7
m_dic['Aug'] = 8
m_dic['Sep'] = 9
m_dic['Oct'] = 10
m_dic['Nov'] = 11
m_dic['Dec'] = 12

In [10]:
curr = datetime(2021,9, 24, 5)

In [48]:
## Find the time difference between the current time and the Twitter post time
weather_related_twitter = []
for twitter in d_twitter:
    loc = twitter[0]
    t = twitter[1].split()
    val = twitter[2]
    month = m_dic[t[1]]
    day = int(t[2])
    year = int(t[5])
    hour = int(t[3].split(":")[0])
    minute = int(t[3].split(":")[1])
    twitter_t = datetime(year, month, day, hour, minute)
    diff_hour = (twitter_t - curr).total_seconds() / (60.0*60)
    if (diff_hour<0 or diff_hour>24):
        #print(diff_hour)
        pass
    else:
        weather_related_twitter.append((loc, diff_hour, val))

In [90]:
t = []
for ele in weather_related_twitter:
    t.append(ele[1])
min(t)

8.933333333333334

In [92]:
t

[20.083333333333332,
 20.1,
 20.1,
 20.133333333333333,
 20.183333333333334,
 20.2,
 20.2,
 20.2,
 20.233333333333334,
 20.283333333333335,
 20.316666666666666,
 20.35,
 20.366666666666667,
 20.4,
 20.416666666666668,
 20.45,
 20.45,
 20.45,
 20.466666666666665,
 20.55,
 20.6,
 20.633333333333333,
 20.683333333333334,
 20.816666666666666,
 20.833333333333332,
 20.85,
 20.85,
 20.9,
 20.95,
 20.983333333333334,
 21.0,
 21.116666666666667,
 21.133333333333333,
 21.183333333333334,
 21.183333333333334,
 21.183333333333334,
 21.2,
 21.216666666666665,
 21.216666666666665,
 21.266666666666666,
 21.316666666666666,
 21.333333333333332,
 21.333333333333332,
 21.383333333333333,
 21.383333333333333,
 21.55,
 21.65,
 21.683333333333334,
 21.716666666666665,
 21.75,
 21.75,
 21.85,
 22.0,
 22.0,
 22.083333333333332,
 22.116666666666667,
 22.116666666666667,
 22.133333333333333,
 22.15,
 22.15,
 22.216666666666665,
 22.216666666666665,
 22.266666666666666,
 22.433333333333334,
 22.5,
 17.61666666

In [50]:
s1 = []
s2 = []
s3 = []
s4 = []
for ele in weather_related_twitter:
    t = ele[1]
    if (t>8 and t<=12):
        s1.append(ele)
    elif (t>12 and t<=16):
        s2.append(ele)
    elif (t>16 and t<=20):
        s3.append(ele)
    elif (t>20 and t<=24):
        s4.append(ele)

In [51]:
len(s1),len(s2),len(s3),len(s4)

(73, 103, 99, 95)

In [53]:
## Generate raster based about 1 km spatial resoltuion 
## one Degree latitude = 111 km
## In chicago, where latitude = 41.881832, one Degree longitude = 82 km
## We use this estimation for the following ananlsis
## This work as the city of Chicago is small
lat_start = 41.05
lon_start = -87.96

incre_lat = 1/111
incre_lon = 1/82

lat_end = 42.05
lon_end = -87.5

raster = []

lat = lat_start

while(lat<lat_end):
    lon = lon_start
    while(lon<lon_end):
        curr_point = Point(lon, lat)
        if (curr_point.within(chicago_shape)):
            raster.append([lon, lat])
        lon = lon+incre_lon
    lat = lat+incre_lat


In [54]:
len(raster)

593

In [55]:
## Generate a random point from a polygon
import random

def generate_random(number, polygon):
    minx, miny, maxx, maxy = polygon.bounds
    pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
    return pnt

In [56]:
## Function to generate random location twitter
## For monte caro experiment
## Enable exact and poly if you want to see how many twitter has exact location and how many comes with a polygon
#exact = 0
#poly = 0
def generate_random_loc(weather_related_twitter):
    random_loc_twitter = []
    for ele in weather_related_twitter:
        loc = ele[0]
        point = 0
        #print(loc)
        if (type(loc)==Point):
            ## exact location extracted
            point = loc
            #exact = exact+1
        else:
            ## Select a random point from a multi-polygon
            point = generate_random(1, Polygon(loc))
            #poly = poly+1
        random_loc_twitter.append([point, ele[2]])
    return random_loc_twitter

In [59]:
## for 4 bins
def get_heat_exposure(weather_related_twitter):
    d_final = {}

    for i in range(0, 100):
        ## try 100 random time
        print("current "+str(i))

        ## Conduct kernel density estimation
        random_loc_twitter = generate_random_loc(weather_related_twitter)

        ### fill with inverse distance weighting


        for ele in raster:
            lon = ele[0]
            lat = ele[1]
            ## iterate through all the values in the existing twitter
            up = 0
            down = 0
            IDW = 0
            for twitter in random_loc_twitter:
                pt = twitter[0]
                curr_x = pt.x
                curr_y = pt.y
                curr_val = twitter[1]

                distx = (curr_x-lon)*82
                disty = (curr_y-lat)*111

                w = 1/np.sqrt(distx*distx+disty*disty)

                down = down+w
                up = up+w*curr_val
            rt = up/down

            key = (ele[0],ele[1])
            if (key not in d_final.keys()):
                d_final[key]=[rt]
            else:
                d_final[key].append(rt)
    return d_final

In [60]:
s1_rt = get_heat_exposure(s1)

current 0
current 1
current 2
current 3
current 4
current 5
current 6
current 7
current 8
current 9
current 10
current 11
current 12
current 13
current 14
current 15
current 16
current 17
current 18
current 19
current 20
current 21
current 22
current 23
current 24
current 25
current 26
current 27
current 28
current 29
current 30
current 31
current 32
current 33
current 34
current 35
current 36
current 37
current 38
current 39
current 40
current 41
current 42
current 43
current 44
current 45
current 46
current 47
current 48
current 49
current 50
current 51
current 52
current 53
current 54
current 55
current 56
current 57
current 58
current 59
current 60
current 61
current 62
current 63
current 64
current 65
current 66
current 67
current 68
current 69
current 70
current 71
current 72
current 73
current 74
current 75
current 76
current 77
current 78
current 79
current 80
current 81
current 82
current 83
current 84
current 85
current 86
current 87
current 88
current 89
current 90
current 9

In [61]:
s2_rt = get_heat_exposure(s2)

current 0
current 1
current 2
current 3
current 4
current 5
current 6
current 7
current 8
current 9
current 10
current 11
current 12
current 13
current 14
current 15
current 16
current 17
current 18
current 19
current 20
current 21
current 22
current 23
current 24
current 25
current 26
current 27
current 28
current 29
current 30
current 31
current 32
current 33
current 34
current 35
current 36
current 37
current 38
current 39
current 40
current 41
current 42
current 43
current 44
current 45
current 46
current 47
current 48
current 49
current 50
current 51
current 52
current 53
current 54
current 55
current 56
current 57
current 58
current 59
current 60
current 61
current 62
current 63
current 64
current 65
current 66
current 67
current 68
current 69
current 70
current 71
current 72
current 73
current 74
current 75
current 76
current 77
current 78
current 79
current 80
current 81
current 82
current 83
current 84
current 85
current 86
current 87
current 88
current 89
current 90
current 9

In [62]:
s3_rt = get_heat_exposure(s3)

current 0
current 1
current 2
current 3
current 4
current 5
current 6
current 7
current 8
current 9
current 10
current 11
current 12
current 13
current 14
current 15
current 16
current 17
current 18
current 19
current 20
current 21
current 22
current 23
current 24
current 25
current 26
current 27
current 28
current 29
current 30
current 31
current 32
current 33
current 34
current 35
current 36
current 37
current 38
current 39
current 40
current 41
current 42
current 43
current 44
current 45
current 46
current 47
current 48
current 49
current 50
current 51
current 52
current 53
current 54
current 55
current 56
current 57
current 58
current 59
current 60
current 61
current 62
current 63
current 64
current 65
current 66
current 67
current 68
current 69
current 70
current 71
current 72
current 73
current 74
current 75
current 76
current 77
current 78
current 79
current 80
current 81
current 82
current 83
current 84
current 85
current 86
current 87
current 88
current 89
current 90
current 9

In [63]:
s4_rt = get_heat_exposure(s4)

current 0
current 1
current 2
current 3
current 4
current 5
current 6
current 7
current 8
current 9
current 10
current 11
current 12
current 13
current 14
current 15
current 16
current 17
current 18
current 19
current 20
current 21
current 22
current 23
current 24
current 25
current 26
current 27
current 28
current 29
current 30
current 31
current 32
current 33
current 34
current 35
current 36
current 37
current 38
current 39
current 40
current 41
current 42
current 43
current 44
current 45
current 46
current 47
current 48
current 49
current 50
current 51
current 52
current 53
current 54
current 55
current 56
current 57
current 58
current 59
current 60
current 61
current 62
current 63
current 64
current 65
current 66
current 67
current 68
current 69
current 70
current 71
current 72
current 73
current 74
current 75
current 76
current 77
current 78
current 79
current 80
current 81
current 82
current 83
current 84
current 85
current 86
current 87
current 88
current 89
current 90
current 9

In [65]:
heat_exposure_map_1 = {}
for key in s1_rt.keys():
    ## Get the average hot exposure
    heat_exposure_map_1[key] = np.mean(s1_rt[key])

In [67]:
heat_exposure_map_2 = {}
for key in s2_rt.keys():
    ## Get the average hot exposure
    heat_exposure_map_2[key] = np.mean(s2_rt[key])

In [68]:
heat_exposure_map_3 = {}
for key in s3_rt.keys():
    ## Get the average hot exposure
    heat_exposure_map_3[key] = np.mean(s3_rt[key])

In [69]:
heat_exposure_map_4 = {}
for key in s4_rt.keys():
    ## Get the average hot exposure
    heat_exposure_map_4[key] = np.mean(s4_rt[key])

In [70]:
mn = min(min(heat_exposure_map_1.values()),min(heat_exposure_map_2.values()),min(heat_exposure_map_3.values()),min(heat_exposure_map_4.values()))

In [72]:
mx = max(max(heat_exposure_map_1.values()),max(heat_exposure_map_2.values()),max(heat_exposure_map_3.values()),max(heat_exposure_map_4.values()))

In [75]:
for key in heat_exposure_map_1.keys():
    norm = (heat_exposure_map_1[key]-mn)/(mx-mn)
    heat_exposure_map_1[key] = norm

In [76]:
for key in heat_exposure_map_2.keys():
    norm = (heat_exposure_map_2[key]-mn)/(mx-mn)
    heat_exposure_map_2[key] = norm

In [77]:
for key in heat_exposure_map_3.keys():
    norm = (heat_exposure_map_3[key]-mn)/(mx-mn)
    heat_exposure_map_3[key] = norm

In [78]:
for key in heat_exposure_map_4.keys():
    norm = (heat_exposure_map_4[key]-mn)/(mx-mn)
    heat_exposure_map_4[key] = norm

In [80]:
final_heat_exposure_map = {}
for key in heat_exposure_map_1.keys():
    val1 = heat_exposure_map_1[key]
    val2 = heat_exposure_map_2[key]
    val3 = heat_exposure_map_3[key]
    val4 = heat_exposure_map_4[key]
    final_heat_exposure_map[key] = [val1, val2, val3, val4]

In [83]:
import csv
with open('./output/heat_exposure_4s.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in final_heat_exposure_map.items():
        writer.writerow([key[0], key[1], value[0], value[1], value[2], value[3] ])